<a href="https://colab.research.google.com/github/sachinsaraff10/March-madness/blob/main/March-Madness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

march_machine_learning_mania_2025_path = kagglehub.competition_download('march-machine-learning-mania-2025')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/march-machine-learning-mania-2025/Conferences.csv
/kaggle/input/march-machine-learning-mania-2025/SeedBenchmarkStage1.csv
/kaggle/input/march-machine-learning-mania-2025/WNCAATourneyDetailedResults.csv
/kaggle/input/march-machine-learning-mania-2025/WRegularSeasonCompactResults.csv
/kaggle/input/march-machine-learning-mania-2025/MNCAATourneySeedRoundSlots.csv
/kaggle/input/march-machine-learning-mania-2025/MRegularSeasonDetailedResults.csv
/kaggle/input/march-machine-learning-mania-2025/MNCAATourneyCompactResults.csv
/kaggle/input/march-machine-learning-mania-2025/MGameCities.csv
/kaggle/input/march-machine-learning-mania-2025/WSecondaryTourneyCompactResults.csv
/kaggle/input/march-machine-learning-mania-2025/WGameCities.csv
/kaggle/input/march-machine-learning-mania-2025/MSeasons.csv
/kaggle/input/march-machine-learning-mania-2025/WNCAATourneySlots.csv
/kaggle/input/march-machine-learning-mania-2025/MSecondaryTourneyTeams.csv
/kaggle/input/march-machine-learning-mania-20

In [ ]:
import cudf  # this should work without any errors

%load_ext cudf.pandas

In [ ]:
path = "/kaggle/input/march-machine-learning-mania-2025/"

In [ ]:
seeds = pd.concat([pd.read_csv(path+"MNCAATourneySeeds.csv").assign(League="M"),
        pd.read_csv(path+"WNCAATourneySeeds.csv").assign(League="W"),],).reset_index(drop=True)

In [ ]:

tournamentres = pd.concat(
    [pd.read_csv(path + "MNCAATourneyDetailedResults.csv").assign(League="M"),
        pd.read_csv(path + "WNCAATourneyDetailedResults.csv").assign(League="W"),]
).reset_index(drop=True)

In [ ]:
regulardetailedres = pd.concat(
    [pd.read_csv(path + "MRegularSeasonDetailedResults.csv").assign(League="M"),
        pd.read_csv(path + "WRegularSeasonDetailedResults.csv").assign(League="W"),]
).reset_index(drop=True)


In [ ]:
detailedcols = regulardetailedres.columns

In [ ]:
def rename_winners(col):
    if col.startswith('W'):
        return col[1:]
    elif col.startswith('L') and col!='League':
        return 'Opp' + col[1:]
    return col

In [ ]:
def rename_losers(col):
    if col.startswith('L') and col!='League':
        return col[1:]
    elif col.startswith('W'):
        return 'Opp' + col[1:]
    return col

In [ ]:
regulardetailedres = pd.concat([
    regulardetailedres[detailedcols].assign(
        Result = 'W')
    .rename(columns=lambda col: rename_winners(col)),
    regulardetailedres[detailedcols].assign(
        Result = 'L')
    .rename(columns=lambda col: rename_losers(col))
])

In [ ]:
regulardetailedres.drop(columns=['Loc','OppLoc'],inplace=True)

In [ ]:
regulardetailedres.isna().sum()

Season       0
DayNum       0
TeamID       0
Score        0
OppTeamID    0
OppScore     0
NumOT        0
FGM          0
FGA          0
FGM3         0
FGA3         0
FTM          0
FTA          0
OR           0
DR           0
Ast          0
TO           0
Stl          0
Blk          0
PF           0
OppFGM       0
OppFGA       0
OppFGM3      0
OppFGA3      0
OppFTM       0
OppFTA       0
OppOR        0
OppDR        0
OppAst       0
OppTO        0
OppStl       0
OppBlk       0
OppPF        0
League       0
Result       0
dtype: int64

In [ ]:
regulardetailedres['Win'] = (regulardetailedres['Result']=='W').astype('int')

In [ ]:
seeds["Seed"]= (
    seeds["Seed"].str.replace("a", "").str.replace("b", "").str[1:].astype("int")
)


In [ ]:
seeds


,Season,Seed,TeamID,League
0,1985,1,1207,M
1,1985,2,1210,M
2,1985,3,1228,M
3,1985,4,1260,M
4,1985,5,1374,M
...,...,...,...,...
4365,2025,12,3193,W
4366,2025,13,3251,W
4367,2025,14,3195,W
4368,2025,15,3117,W


In [ ]:
regulardetailedres['FG_Percent'] = ((regulardetailedres['FGM'] / regulardetailedres['FGA']) * 100).round(2)
regulardetailedres['ThreeFG_Percent'] =( ((regulardetailedres['FGM3'] / regulardetailedres['FGA3']).replace(np.nan,0)) * 100).round(2)
regulardetailedres['Total_Rebounds'] = regulardetailedres['OR'] + regulardetailedres['DR']
regulardetailedres['AST_TO_Ratio'] = regulardetailedres['Ast'] / regulardetailedres['TO']
regulardetailedres['FT_Percent'] = ( ((regulardetailedres['FGM3'] / regulardetailedres['FGA3']).replace(np.nan,0)) * 100).round(2)

In [ ]:
regularaggregates =( regulardetailedres.groupby(['Season','TeamID','League'])
.agg(
        FG_Percent=('FG_Percent', 'mean'),
        ThreeFG_Percent=('ThreeFG_Percent', 'mean'),
        Mean_Rebounds=('Total_Rebounds', 'mean'),
        Median_Turnovers=('TO', 'median'),
        Median_Fouls=('PF', 'median'),
        AST_TO_Ratio=('AST_TO_Ratio', 'mean'),
        FT_Percent=('FT_Percent', 'mean'),
        Mean_FTM=('FTM', 'mean'),
        Win_Percent=('Win', lambda x: (x.sum() / len(x)) * 100),
)
 .reset_index())

In [ ]:
columns_to_round = [
    'FG_Percent', 'ThreeFG_Percent', 'FT_Percent',
    'AST_TO_Ratio', 'Mean_FTM', 'Win_Percent','Mean_Rebounds'
]

regularaggregates[columns_to_round] = regularaggregates[columns_to_round].round(2)


In [ ]:
regularaggregates.shape

(13583, 12)

In [ ]:
seeds.query('Season == 2025').nunique()

Season      1
Seed       16
TeamID    136
League      2
dtype: int64

In [ ]:
regular_agg_final = regularaggregates.merge(seeds,
                                            on=["Season", "TeamID", "League"],
                                            how="inner")

In [ ]:
regular_agg_final.isna().sum()


Season              0
TeamID              0
League              0
FG_Percent          0
ThreeFG_Percent     0
Mean_Rebounds       0
Median_Turnovers    0
Median_Fouls        0
AST_TO_Ratio        0
FT_Percent          0
Mean_FTM            0
Win_Percent         0
Seed                0
dtype: int64

In [ ]:
allcols = regular_agg_final.columns

In [ ]:
tournamentres=pd.concat([
    tournamentres[detailedcols].assign(
        Result = 'W')
    .rename(columns=lambda col: rename_winners(col)),
    tournamentres[detailedcols].assign(
        Result = 'L')
    .rename(columns=lambda col: rename_losers(col))
])

In [ ]:
tournamentres.drop(columns=['Loc','OppLoc'],inplace=True)

In [ ]:
tournamentres.columns

Index(['Season', 'DayNum', 'TeamID', 'Score', 'OppTeamID', 'OppScore', 'NumOT',
       'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO',
       'Stl', 'Blk', 'PF', 'OppFGM', 'OppFGA', 'OppFGM3', 'OppFGA3', 'OppFTM',
       'OppFTA', 'OppOR', 'OppDR', 'OppAst', 'OppTO', 'OppStl', 'OppBlk',
       'OppPF', 'League', 'Result'],
      dtype='object')

In [ ]:
def oppnamer(col):
    return 'Opp'+col if (col!='League') and (col!='Season') else col

In [ ]:
allcols

Index(['Season', 'TeamID', 'League', 'FG_Percent', 'ThreeFG_Percent',
       'Mean_Rebounds', 'Median_Turnovers', 'Median_Fouls', 'AST_TO_Ratio',
       'FT_Percent', 'Mean_FTM', 'Win_Percent', 'Seed'],
      dtype='object')

In [ ]:
tournament_final = tournamentres.merge(
    regular_agg_final[allcols],
    on = ['Season','League','TeamID'],
    how='left',
).merge(
    regular_agg_final[allcols].rename(
        columns = lambda col: oppnamer(col)
    ),
    on=["Season", "League", "OppTeamID"],
)

In [ ]:
tournament_final.columns

Index(['Season', 'DayNum', 'TeamID', 'Score', 'OppTeamID', 'OppScore', 'NumOT',
       'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO',
       'Stl', 'Blk', 'PF', 'OppFGM', 'OppFGA', 'OppFGM3', 'OppFGA3', 'OppFTM',
       'OppFTA', 'OppOR', 'OppDR', 'OppAst', 'OppTO', 'OppStl', 'OppBlk',
       'OppPF', 'League', 'Result', 'FG_Percent', 'ThreeFG_Percent',
       'Mean_Rebounds', 'Median_Turnovers', 'Median_Fouls', 'AST_TO_Ratio',
       'FT_Percent', 'Mean_FTM', 'Win_Percent', 'Seed', 'OppFG_Percent',
       'OppThreeFG_Percent', 'OppMean_Rebounds', 'OppMedian_Turnovers',
       'OppMedian_Fouls', 'OppAST_TO_Ratio', 'OppFT_Percent', 'OppMean_FTM',
       'OppWin_Percent', 'OppSeed'],
      dtype='object')

In [ ]:
tournament_final.query('TeamID == 1421 or TeamID == 1411')

,Season,DayNum,TeamID,Score,OppTeamID,OppScore,NumOT,FGM,FGA,FGM3,...,OppFG_Percent,OppThreeFG_Percent,OppMean_Rebounds,OppMedian_Turnovers,OppMedian_Fouls,OppAST_TO_Ratio,OppFT_Percent,OppMean_FTM,OppWin_Percent,OppSeed
0,2003,134,1421,92,1411,84,1,32,69,11,...,44.89,32.14,37.97,15.0,19.0,0.98,32.14,17.40,60.00,16
513,2011,134,1421,81,1114,77,1,27,54,4,...,41.52,39.24,32.48,12.0,20.0,1.05,39.24,15.64,51.52,16
984,2018,135,1411,64,1300,46,0,21,57,6,...,46.20,34.58,36.61,13.0,19.0,1.19,34.58,12.10,51.61,16
1117,2021,136,1411,60,1291,52,0,22,57,6,...,42.75,33.70,34.09,13.0,14.5,1.05,33.70,10.91,54.55,16
1182,2022,134,1411,76,1394,67,0,23,55,4,...,42.61,31.65,34.33,13.0,22.0,1.03,31.65,17.20,63.33,16
2276,2003,134,1411,84,1421,92,1,29,67,12,...,43.28,36.35,35.45,16.0,19.0,0.92,36.35,15.97,44.83,16
2305,2003,137,1421,61,1400,82,0,23,62,5,...,44.89,34.92,42.32,13.5,20.0,1.13,34.92,17.00,78.57,1
2801,2011,136,1421,51,1338,74,0,17,55,3,...,47.29,38.33,39.88,11.0,18.0,1.75,38.33,16.00,84.38,1
2871,2012,136,1421,65,1393,72,0,21,49,9,...,46.50,33.62,35.33,9.0,15.0,1.66,33.62,13.03,93.94,1
2991,2014,135,1411,69,1142,81,0,24,50,4,...,40.51,33.38,31.47,9.0,17.0,1.48,33.38,12.20,36.67,16


In [ ]:
team_names = pd.concat([pd.read_csv(path+"MTeams.csv").assign(League="M"),
        pd.read_csv(path+"WTeams.csv").assign(League="W"),],).reset_index(drop=True)

In [ ]:
team_names.drop('FirstD1Season',axis=1,inplace=True)

In [ ]:
team_names.shape

(758, 4)

In [ ]:
team_names.drop('LastD1Season',axis=1,inplace=True)


In [ ]:
tournament_v2 = tournament_final.merge(
    team_names,
    on=['TeamID','League'],
    how = 'left'
)

Model 1: Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import GroupKFold

In [ ]:
tournament_v2.columns

Index(['Season', 'DayNum', 'TeamID', 'Score', 'OppTeamID', 'OppScore', 'NumOT',
       'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO',
       'Stl', 'Blk', 'PF', 'OppFGM', 'OppFGA', 'OppFGM3', 'OppFGA3', 'OppFTM',
       'OppFTA', 'OppOR', 'OppDR', 'OppAst', 'OppTO', 'OppStl', 'OppBlk',
       'OppPF', 'League', 'Result', 'FG_Percent', 'ThreeFG_Percent',
       'Mean_Rebounds', 'Median_Turnovers', 'Median_Fouls', 'AST_TO_Ratio',
       'FT_Percent', 'Mean_FTM', 'Win_Percent', 'Seed', 'OppFG_Percent',
       'OppThreeFG_Percent', 'OppMean_Rebounds', 'OppMedian_Turnovers',
       'OppMedian_Fouls', 'OppAST_TO_Ratio', 'OppFT_Percent', 'OppMean_FTM',
       'OppWin_Percent', 'OppSeed', 'TeamName'],
      dtype='object')

In [ ]:
tournament_v2

,Season,DayNum,TeamID,Score,OppTeamID,OppScore,NumOT,FGM,FGA,FGM3,...,OppThreeFG_Percent,OppMean_Rebounds,OppMedian_Turnovers,OppMedian_Fouls,OppAST_TO_Ratio,OppFT_Percent,OppMean_FTM,OppWin_Percent,OppSeed,TeamName
0,2003,134,1421,92,1411,84,1,32,69,11,...,32.14,37.97,15.0,19.0,0.98,32.14,17.40,60.00,16,UNC Asheville
1,2003,136,1112,80,1436,51,0,31,66,7,...,35.79,38.69,14.0,16.0,1.08,35.79,12.86,65.52,16,Arizona
2,2003,136,1113,84,1272,71,0,31,59,6,...,34.34,40.03,13.0,18.0,1.29,34.34,14.97,79.31,7,Arizona St
3,2003,136,1141,79,1166,73,0,29,53,3,...,38.49,34.06,13.0,17.0,1.33,38.49,13.88,87.88,6,C Michigan
4,2003,136,1143,76,1301,74,1,27,64,7,...,35.06,31.77,14.0,18.0,1.10,35.06,15.77,60.00,9,California
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4547,2024,147,3425,73,3163,80,0,23,70,9,...,35.82,33.39,12.0,15.0,1.72,35.82,11.70,84.85,3,USC
4548,2024,147,3261,87,3234,94,0,34,88,8,...,37.49,37.36,14.0,15.0,1.82,37.49,15.30,87.88,1,LSU
4549,2024,151,3163,69,3234,71,0,29,63,8,...,37.49,37.36,14.0,15.0,1.82,37.49,15.30,87.88,1,Connecticut
4550,2024,151,3301,59,3376,78,0,20,62,6,...,39.79,42.12,13.0,15.0,1.57,39.79,13.16,100.00,1,NC State


In [ ]:
# tournament_v2['Rebound_diff'] = (tournament_v2.OR + tournament_v2.DR)-(tournament_v2.OppOR + tournament_v2.OppDR)

In [ ]:
# tournament_v2['3pointdiff'] = tournament_v2.FGM3 - tournament_v2.OppFGM3
# tournament_v2['FGdiff'] = tournament_v2.FGM - tournament_v2.OppFGM
# tournament_v2['TOdiff'] = tournament_v2.TO - tournament_v2.OppTO
# tournament_v2['Fouldiff'] = tournament_v2.PF - tournament_v2.OppPF



In [ ]:
tournament_v2.columns

Index(['Season', 'DayNum', 'TeamID', 'Score', 'OppTeamID', 'OppScore', 'NumOT',
       'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO',
       'Stl', 'Blk', 'PF', 'OppFGM', 'OppFGA', 'OppFGM3', 'OppFGA3', 'OppFTM',
       'OppFTA', 'OppOR', 'OppDR', 'OppAst', 'OppTO', 'OppStl', 'OppBlk',
       'OppPF', 'League', 'Result', 'FG_Percent', 'ThreeFG_Percent',
       'Mean_Rebounds', 'Median_Turnovers', 'Median_Fouls', 'AST_TO_Ratio',
       'FT_Percent', 'Mean_FTM', 'Win_Percent', 'Seed', 'OppFG_Percent',
       'OppThreeFG_Percent', 'OppMean_Rebounds', 'OppMedian_Turnovers',
       'OppMedian_Fouls', 'OppAST_TO_Ratio', 'OppFT_Percent', 'OppMean_FTM',
       'OppWin_Percent', 'OppSeed', 'TeamName'],
      dtype='object')

In [ ]:
features=['FG_Percent', 'ThreeFG_Percent','Mean_Rebounds','Median_Turnovers',
          'Median_Fouls','AST_TO_Ratio','FT_Percent','Mean_FTM','Win_Percent',
          'Seed','OppFG_Percent','OppThreeFG_Percent','OppMean_Rebounds',
          'OppMedian_Turnovers','OppMedian_Fouls','OppAST_TO_Ratio','OppFT_Percent',
          'OppMean_FTM','OppWin_Percent','OppSeed']

In [ ]:
tournament_v2['Win'] = (tournament_v2.Result == 'W').astype('int')

In [ ]:
X = tournament_v2[features]
y= tournament_v2.Win

In [ ]:
folds= GroupKFold(n_splits=tournament_v2.Season.nunique())
crossval =[]
models = []

In [ ]:
groups= tournament_v2.Season
seasons=groups.unique()

In [ ]:
holdout = 0
for train_index, test_index in folds.split(X, y, groups):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Prepare the model
    model = RandomForestClassifier(
        n_estimators=100,  # Number of trees
        max_depth=None,    # Allow trees to grow fully
        random_state=42
    )
    holdout_season = seasons[holdout]
    print(f"Holdout Season: {holdout_season}")
    # Train the model
    model.fit(X_train, y_train)

    # Predict probabilities for log loss
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    score_ll = log_loss(y_test, y_pred_proba)

    # Predict class labels for accuracy
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    # Track results
    crossval.append(accuracy)
    holdout += 1
    print(f"Season {holdout_season}: Accuracy = {accuracy}, Log Loss = {score_ll}")
    models.append(model)

# Print the average accuracy across all folds
print("Average CV Accuracy:", np.mean(crossval))


Holdout Season: 2003
Season 2003: Accuracy = 0.7611940298507462, Log Loss = 0.5013563205717246
Holdout Season: 2004
Season 2004: Accuracy = 0.7126865671641791, Log Loss = 0.576343373755956
Holdout Season: 2005
Season 2005: Accuracy = 0.7126865671641791, Log Loss = 0.5730618947313261
Holdout Season: 2006
Season 2006: Accuracy = 0.7461538461538462, Log Loss = 0.4879891900940406
Holdout Season: 2007
Season 2007: Accuracy = 0.7192307692307692, Log Loss = 0.5383207059481757
Holdout Season: 2008
Season 2008: Accuracy = 0.7307692307692307, Log Loss = 0.5286796478864159
Holdout Season: 2009
Season 2009: Accuracy = 0.7384615384615385, Log Loss = 0.5145633769067902
Holdout Season: 2010
Season 2010: Accuracy = 0.7038461538461539, Log Loss = 0.5570505088150675
Holdout Season: 2011
Season 2011: Accuracy = 0.7846153846153846, Log Loss = 0.47808337677723917
Holdout Season: 2012
Season 2012: Accuracy = 0.7807692307692308, Log Loss = 0.5052050613432724
Holdout Season: 2013
Season 2013: Accuracy = 0.730

In [ ]:
importance = model.feature_importances_

# Display importance as a DataFrame
importance_df = pd.DataFrame({
    "Feature": features,
    "Importance": importance
}).sort_values(by="Importance", ascending=False)

In [ ]:
importance_df

,Feature,Importance
19,OppSeed,0.124199
9,Seed,0.117133
8,Win_Percent,0.057948
18,OppWin_Percent,0.056893
0,FG_Percent,0.056464
10,OppFG_Percent,0.056013
2,Mean_Rebounds,0.047060
5,AST_TO_Ratio,0.046340
15,OppAST_TO_Ratio,0.046336
12,OppMean_Rebounds,0.045469


In [ ]:
import xgboost as xgboost

In [ ]:
seeds25=seeds.query('Season==2025')

In [ ]:
opps25 = seeds25.rename(columns=lambda col: f"Opp{col}" if col != "League" and col != "Season" else col)


In [ ]:
pairs = (
    seeds25.merge(opps25,
                 on=['League','Season']
                 )
    .query('TeamID!=OppTeamID')
)

In [ ]:
pairs = (pairs.merge(
    regular_agg_final[allcols],
    on = ['Season','League','TeamID','Seed'],
    how='left',
).merge(
    regular_agg_final[allcols].rename(
        columns = lambda col: oppnamer(col)
    ),
    on=["Season", "League", "OppTeamID",'OppSeed'],
).reset_index(drop=True))

In [ ]:
pairs.isna().sum()

Season                 0
Seed                   0
TeamID                 0
League                 0
OppSeed                0
OppTeamID              0
FG_Percent             0
ThreeFG_Percent        0
Mean_Rebounds          0
Median_Turnovers       0
Median_Fouls           0
AST_TO_Ratio           0
FT_Percent             0
Mean_FTM               0
Win_Percent            0
OppFG_Percent          0
OppThreeFG_Percent     0
OppMean_Rebounds       0
OppMedian_Turnovers    0
OppMedian_Fouls        0
OppAST_TO_Ratio        0
OppFT_Percent          0
OppMean_FTM            0
OppWin_Percent         0
dtype: int64

In [ ]:
models

[RandomForestClassifier(random_state=42),
 RandomForestClassifier(random_state=42),
 RandomForestClassifier(random_state=42),
 RandomForestClassifier(random_state=42),
 RandomForestClassifier(random_state=42),
 RandomForestClassifier(random_state=42),
 RandomForestClassifier(random_state=42),
 RandomForestClassifier(random_state=42),
 RandomForestClassifier(random_state=42),
 RandomForestClassifier(random_state=42),
 RandomForestClassifier(random_state=42),
 RandomForestClassifier(random_state=42),
 RandomForestClassifier(random_state=42),
 RandomForestClassifier(random_state=42),
 RandomForestClassifier(random_state=42),
 RandomForestClassifier(random_state=42),
 RandomForestClassifier(random_state=42),
 RandomForestClassifier(random_state=42),
 RandomForestClassifier(random_state=42),
 RandomForestClassifier(random_state=42),
 RandomForestClassifier(random_state=42)]

In [ ]:
pairs['pred'] = 0

In [ ]:
# Replace infinite values with NaN
pairs[features] = pairs[features].replace([np.inf, -np.inf], np.nan)


In [ ]:
pairs[features]=pairs[features].fillna(0)

In [ ]:
for model in models:
    pairs['pred'] += model.predict_proba(pairs[features])[:,1]

In [ ]:
pairs['pred']/=len(models)

In [ ]:
pairs.pred = pairs.pred.round(2)

In [ ]:
pairs["ID"] = (
    pairs["Season"].astype("str")+ "_"
    + pairs["TeamID"].astype("str")
    + "_"+ pairs["OppTeamID"].astype("str")
)


In [ ]:
# Bracket simulation-
# Code used from https://www.kaggle.com/code/lennarthaupts/simulate-n-brackets

from tqdm import tqdm

# Load and filter data
round_slots = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MNCAATourneySlots.csv')
round_slots = round_slots[round_slots['Season'] == 2025]
round_slots = round_slots[round_slots['Slot'].str.contains('R')] # Filter out First Four


In [ ]:
seeds25 = pd.concat([pd.read_csv(path+"MNCAATourneySeeds.csv").assign(League="M"),
        pd.read_csv(path+"WNCAATourneySeeds.csv").assign(League="W"),],).query('Season==2025').reset_index(drop=True)

In [ ]:
seeds25

,Season,Seed,TeamID,League
0,2025,W01,1181,M
1,2025,W02,1104,M
2,2025,W03,1458,M
3,2025,W04,1112,M
4,2025,W05,1332,M
...,...,...,...,...
131,2025,Z12,3193,W
132,2025,Z13,3251,W
133,2025,Z14,3195,W
134,2025,Z15,3117,W


In [ ]:
seeds25.Seed.str

In [ ]:
seeds25['Seed'] = (seeds25['Seed'].str.replace('a','').str.replace('b',''))

In [ ]:
seeds_m=seeds25[seeds25.League == 'M']
seeds_w=seeds25[seeds25.League == 'W']

In [ ]:
pairs['ID'] = pairs['ID'].str.split('_')

In [ ]:
pairs['ID']

0       [2025, 1181, 1104]
1       [2025, 1181, 1458]
2       [2025, 1181, 1112]
3       [2025, 1181, 1332]
4       [2025, 1181, 1140]
               ...        
9107    [2025, 3422, 3293]
9108    [2025, 3422, 3193]
9109    [2025, 3422, 3251]
9110    [2025, 3422, 3195]
9111    [2025, 3422, 3117]
Name: ID, Length: 9112, dtype: object

In [ ]:
def prepare_data(seeds, preds):
    # Function preparing the data for the simulation
    seed_dict = seeds.set_index("Seed")["TeamID"].to_dict()
    inverted_seed_dict = {value: key for key, value in seed_dict.items()}
    probas_dict = {}

    for teams, proba in zip(preds["ID"], preds["pred"]):
        team1, team2 = teams[1], teams[2]
        # print(team1,team2)
        probas_dict.setdefault(team1, {})[team2] = proba
        probas_dict.setdefault(team2, {})[team1] = 1 - proba
    # print(teams,team1)
    return seed_dict, inverted_seed_dict, probas_dict


def simulate(round_slots, seeds, inverted_seeds, probas, sim=True):
    """
    Simulates each round of the tournament.

    Parameters:
    - round_slots: DataFrame containing information on who is playing in each round.
    - seeds (dict): Dictionary mapping seed values to team IDs.
    - inverted_seeds (dict): Dictionary mapping team IDs to seed values.
    - probas (dict): Dictionary containing matchup probabilities.
    - sim (boolean): Simulates match if True. Chooses team with higher probability as winner otherwise.

    Returns:
    - list: List with winning team IDs for each match.
    - list: List with corresponding slot names for each match.
    """
    winners = []
    slots = []
    # print(seeds)
    for slot, strong, weak in zip(
        round_slots.Slot, round_slots.StrongSeed, round_slots.WeakSeed
    ):
        team_1, team_2 = seeds[strong], seeds[weak]
        # print(team_1,team_2)
        # print(seeds)
        # Get the probability of team_1 winning
        # print(probas)
        proba = probas[str(team_1)][str(team_2)]

        if sim:
            # Randomly determine the winner based on the probability
            winner = np.random.choice([team_1, team_2], p=[proba, 1 - proba])
        else:
            # Determine the winner based on the higher probability
            winner = [team_1, team_2][np.argmax([proba, 1 - proba])]

        # Append the winner and corresponding slot to the lists
        winners.append(winner)
        slots.append(slot)

        seeds[slot] = winner

    # Convert winners to original seeds using the inverted_seeds dictionary
    return [inverted_seeds[w] for w in winners], slots


def run_simulation(brackets=1, seeds=None, preds=None, round_slots=None, sim=True):
    """
    Runs a simulation of bracket tournaments.

    Parameters:
    - brackets (int): Number of brackets to simulate.
    - seeds (pd.DataFrame): DataFrame containing seed information.
    - preds (pd.DataFrame): DataFrame containing prediction information for each match-up.
    - round_slots (pd.DataFrame): DataFrame containing information about the tournament rounds.
    - sim (boolean): Simulates matches if True. Chooses team with higher probability as winner otherwise.

    Returns:
    - pd.DataFrame: DataFrame with simulation results.
    """
    # Get relevant data for the simulation
    seed_dict, inverted_seed_dict, probas_dict = prepare_data(seeds, preds)
    # Lists to store simulation results
    results = []
    bracket = []
    slots = []

    # Iterate through the specified number of brackets
    for b in tqdm(range(1, brackets + 1)):
        # Run single simulation
        r, s = simulate(round_slots, seed_dict, inverted_seed_dict, probas_dict, sim)

        # Update results
        results.extend(r)
        bracket.extend([b] * len(r))
        slots.extend(s)

    # Create final DataFrame
    result_df = pd.DataFrame({"Bracket": bracket, "Slot": slots, "Team": results})

    return result_df


n_brackets = 1000
result_m = run_simulation(
    brackets=n_brackets, seeds=seeds_m, preds=pairs, round_slots=round_slots, sim=False
)
result_m["League"] = "M"
result_w = run_simulation(
    brackets=n_brackets, seeds=seeds_w, preds=pairs, round_slots=round_slots, sim=False
)
result_w["League"] = "W"
sim_df = pd.concat([result_m, result_w])
sim_df.reset_index(drop=True,inplace=True)


100%|██████████| 1000/1000 [00:00<00:00, 2927.23it/s]


In [ ]:
sim_df

,Bracket,Slot,Team,League
0,1,R1W1,W01,M
1,1,R1W2,W02,M
2,1,R1W3,W03,M
3,1,R1W4,W04,M
4,1,R1W5,W05,M
...,...,...,...,...
125995,1000,R4Y1,Y01,W
125996,1000,R4Z1,Z02,W
125997,1000,R5WX,X02,W
125998,1000,R5YZ,Z02,W


In [ ]:
!git clone https://github.com/sachinsaraff10/March-madness.git

Cloning into 'March-madness'...


In [3]:
!pwd

/content
